## Imports

In [6]:
### Required L4 GPU at the very least
## Note image outputs have been truncated

In [ ]:
!pip install nibabel -q
!pip install pybboxes -q
!pip install sahi -q

!pip install -i https://test.pypi.org/simple/ supervision==0.3.0
!pip install -q transformers
!pip install -q pytorch-lightning
!pip install -q roboflow
!pip install -q timm -q
!pip install pycocotools -q
!pip install opencv-python -q
!pip install -i https://test.pypi.org/simple/ supervision==0.3.0

!pip install tensorboard -q

!pip install --upgrade torch torchvision
## Reference : https://github.com/roboflow/notebooks/blob/main/notebooks/train-huggingface-detr-on-custom-dataset.ipynb


In [ ]:
import pathlib as plb
import tempfile
import os

import shutil
import pandas as pd


import nibabel as nib
import numpy as np

import sys
import shutil

import pandas as pd
from tqdm import tqdm
from PIL import Image

import enum
import time
import random
import multiprocessing
from pathlib import Path

import torch
import torchvision

import torch.nn.functional as F
import torchvision.transforms as T

import numpy as np
# from unet import UNet
from scipy import stats
import matplotlib.pyplot as plt

from IPython import display
from tqdm.auto import tqdm
from pathlib import Path
import pandas as pd
import os
from torchvision.utils import save_image
from skimage.measure import regionprops_table
from sklearn.model_selection import train_test_split

import wandb

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
## Checkpoint
CHECKPOINT_PATH = '/mnt/disks/storage/vk2501/checkpoints'

## Base Path
BASE_PATH = '/mnt/disks/storage/vk2501/data/PETCT_stacked_DETR'

## DETR Path
DETR = '/mnt/disks/storage/vk2501/data/PETCT_stacked_DETR'

## Dataset for actual file split
COMMON_DATASET = '/mnt/disks/storage/vk2501/data/DETR_labels_27nov'

## Dimensions of each image
DIM = 408

## Environment setup

In [ ]:
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)


import supervision
import transformers
import pytorch_lightning

print(

    "; supervision:", supervision.__version__,
    "; transformers:", transformers.__version__,
    "; pytorch_lightning:", pytorch_lightning.__version__
)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
; supervision: 0.3.0 ; transformers: 4.35.2 ; pytorch_lightning: 2.1.2


### View Data

In [ ]:
## Use Only the required columns
columns= ["img_filename","x_min","y_min","x_max","y_max","cancer_type","img_width","img_height"]
columns_rename = {'img_filename': 'slice'}
COMMON_DATASET

'/mnt/disks/storage/vk2501/data/DETR_labels_27nov'

In [ ]:
import torch
from transformers import DetrForObjectDetection, DetrImageProcessor
from transformers import ConditionalDetrForObjectDetection, ConditionalDetrImageProcessor

# Read csv files containg image fil names for each of the split
train_df = pd.read_csv(COMMON_DATASET + '/train.csv',usecols=columns)
train_df.rename(columns =columns_rename,  inplace=True)
train_set = train_df['slice'].tolist()

validation_df = pd.read_csv(COMMON_DATASET + '/val.csv')
validation_df.rename(columns =columns_rename,  inplace=True)
validation_set = validation_df['slice'].tolist()

test_df = pd.read_csv(COMMON_DATASET + '/test.csv')
test_df.rename(columns =columns_rename,  inplace=True)
test_set = test_df['slice'].tolist()

## Set CUDA for GPU
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Switch based on which model we're using Facebook DETR vs Microsoft DETR
# CHECKPOINT = 'facebook/detr-resnet-50'
CHECKPOINT = 'microsoft/conditional-detr-resnet-50'
CONFIDENCE_TRESHOLD = 0.5
IOU_TRESHOLD =0.5

image_processor = ConditionalDetrImageProcessor.from_pretrained(CHECKPOINT)
model = ConditionalDetrForObjectDetection.from_pretrained(CHECKPOINT)

model.to(DEVICE)
print("loaded")

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


loaded


In [ ]:
df2 = pd.read_csv('/mnt/disks/storage/vk2501/data/master_labels.csv')
print(len(df2[df2['split']=='train']))

print(len(train_set))

19753
14753


In [ ]:
import os
print(DETR)
folders = ['train' ,'test', 'val']
for folder in folders :

    folder_path = DETR + "/" + str(folder)

    # Get the list of all files in the folder
    file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    print(f'length of files in {folder} : {len(file_names)}')

/mnt/disks/storage/vk2501/data/PETCT_stacked_DETR
length of files in train : 19756
length of files in test : 3957
length of files in val : 7916


### View existing files

In [ ]:
import cv2
import torch
import supervision as sv

### Run Model on Existing Images

In [4]:
CONFIDENCE_TRESHOLD=0.3

print("Confidence Threshold : ", CONFIDENCE_TRESHOLD)

IMAGE_PATH = DETR  + '/train/' + '<Image_Name>'

with torch.no_grad():

    # load image and predict
    image = cv2.imread(IMAGE_PATH)
    inputs = image_processor(images=image, return_tensors='pt').to(DEVICE)
    outputs = model(**inputs)

    # Take scores of above CONFIDENCE_TRESHOLD
    target_sizes = torch.tensor([image.shape[:2]]).to(DEVICE)
    results = image_processor.post_process_object_detection(
        outputs=outputs,
        threshold=CONFIDENCE_TRESHOLD,
        target_sizes=target_sizes
    )[0]

# Add Annotations
detections = sv.Detections.from_transformers(transformers_results=results)

labels = [
    f"{model.config.id2label[class_id]} {confidence:0.2f}"
    for _, confidence, class_id, _
    in detections
]

box_annotator = sv.BoxAnnotator()
frame = box_annotator.annotate(scene=image, detections=detections, labels=labels)

%matplotlib inline
sv.show_frame_in_notebook(frame, (4, 4))
# plt.imshow(frame)  # 'cmap' specifies the color map (here, grayscale)

# plt.show()

## Create COCO data loaders

In [ ]:
!cp /mnt/disks/storage/vk2501/data/annotations_28_NOV_stacked_detr/train/_annotations_stacked_all.coco.json  /mnt/disks/storage/vk2501/data/PETCT_stacked_DETR/train
!cp /mnt/disks/storage/vk2501/data/annotations_28_NOV_stacked_detr/test/_annotations_stacked_all.coco.json   /mnt/disks/storage/vk2501/data/PETCT_stacked_DETR/test
!cp /mnt/disks/storage/vk2501/data/annotations_28_NOV_stacked_detr/val/_annotations_stacked_all.coco.json    /mnt/disks/storage/vk2501/data/PETCT_stacked_DETR/val

!cp /mnt/disks/storage/vk2501/data/annotations_28_NOV_stacked_detr/train/_annotations_stacked_positive.coco.json  /mnt/disks/storage/vk2501/data/PETCT_stacked_DETR/train
!cp /mnt/disks/storage/vk2501/data/annotations_28_NOV_stacked_detr/test/_annotations_stacked_positive.coco.json   /mnt/disks/storage/vk2501/data/PETCT_stacked_DETR/test
!cp /mnt/disks/storage/vk2501/data/annotations_28_NOV_stacked_detr/val/_annotations_stacked_positive.coco.json    /mnt/disks/storage/vk2501/data/PETCT_stacked_DETR/val

In [ ]:
import os
import torchvision


ANNOTATION_FILE_NAME = "_annotations_stacked_only_positive.coco.json"

ANNOTATION_FILE_NAME = "_annotations_stacked_all.coco.json"


TRAIN_DIRECTORY = os.path.join(DETR, "train")
VAL_DIRECTORY = os.path.join(DETR, "val")
TEST_DIRECTORY = os.path.join(DETR, "test")


class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(
        self,
        image_directory_path: str,
        image_processor,
        train: bool = True
    ):
        annotation_file_path = os.path.join(image_directory_path, ANNOTATION_FILE_NAME)
        super(CocoDetection, self).__init__(image_directory_path, annotation_file_path)
        self.image_processor = image_processor
        self.file_names = [item['file_name'] for item in self.coco.dataset['images']]

    def get_file_names(self):
        return self.file_names


    def __getitem__(self, idx):
        images, annotations = super(CocoDetection, self).__getitem__(idx)
        image_id = self.ids[idx]
        annotations = {'image_id': image_id, 'annotations': annotations}
        encoding = self.image_processor(images=images, annotations=annotations, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze()
        target = encoding["labels"][0]

        return pixel_values, target


TRAIN_DATASET = CocoDetection(
    image_directory_path=TRAIN_DIRECTORY,
    image_processor=image_processor,
    train=True)
VAL_DATASET = CocoDetection(
    image_directory_path=VAL_DIRECTORY,
    image_processor=image_processor,
    train=False)
TEST_DATASET = CocoDetection(
    image_directory_path=TEST_DIRECTORY,
    image_processor=image_processor,
    train=False)

print("Number of training examples:", len(TRAIN_DATASET))
print("Number of validation examples:", len(VAL_DATASET))
print("Number of test examples:", len(TEST_DATASET))

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Number of training examples: 19751
Number of validation examples: 7913
Number of test examples: 3954


In [ ]:
import os
print(DETR)
folders = ['train' ,'test', 'val']

train_file_names = TRAIN_DATASET.get_file_names()

test_file_names = TEST_DATASET.get_file_names()
val_file_names = VAL_DATASET.get_file_names()

files = [train_file_names, test_file_names, val_file_names]

for idx,folder in enumerate(folders) :

    folder_path = DETR + "/" + str(folder)

    # Get the list of all files in the folder
    file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]


    zero_byte_files = [file for file in file_names if os.path.getsize(os.path.join(folder_path, file)) == 0]

    train_files_not_in_actual = set(files[idx]) - set(file_names)
    print(folder)

    print(len(file_names))
    print(len(files[idx]))

    print((zero_byte_files))

    assert len(train_files_not_in_actual) == 0



/mnt/disks/storage/vk2501/data/PETCT_stacked_DETR
train
19756
19751
['PETCT_d69c3fceba_PETCTstacked_axial_292.jpg', 'PETCT_36d8219e3f_PETCTstacked_axial_700.jpg']
test
3957
3954
[]
val
7916
7913
[]


## Visualize data entry

**NOTE:** Feel free to reload this cell multiple times. Notebook should display different train set image each time.

In [ ]:
DETR

'/mnt/disks/storage/vk2501/data/PETCT_stacked_DETR'

In [3]:
import random
import cv2
import numpy as np

for i in range(0,3):

        positive = True
        # select random image
        image_ids = TRAIN_DATASET.coco.getImgIds()

        image_id = random.choice(image_ids)
        print('Image #{}'.format(image_id))

        # load image and annotatons
        image = TRAIN_DATASET.coco.loadImgs(image_id)[0]
        annotations = TRAIN_DATASET.coco.imgToAnns[image_id]
        image_path = os.path.join(TRAIN_DATASET.root, image['file_name'])
        image = cv2.imread(image_path)

        # annotate
        if len(annotations) > 0:
            detections = sv.Detections.from_coco_annotations(coco_annotation=annotations)

            # we will use id2label function for training
            categories = TRAIN_DATASET.coco.cats
            id2label = {k: v['name'] for k,v in categories.items()}

            labels = [
                f"{id2label[class_id]}"
                for _, _, class_id, _
                in detections
            ]

            box_annotator = sv.BoxAnnotator()
            frame = box_annotator.annotate(scene=image, detections=detections, labels=labels)

            %matplotlib inline
            sv.show_frame_in_notebook(image, (4, 4))

        else:
                positive=False
                print(f'{20*4} Negative Slice ')

                sv.show_frame_in_notebook(image, (4, 4))
#
        print( str(34*'#') + " Original Image Assertion " + str(34*'#') )


        ######################################################################################################################

        import json

        # Specify the file path
        file_path = DETR + "/train/"+ ANNOTATION_FILE_NAME

        # Read the JSON file
        with open(file_path) as file:
            images_data = json.load(file)
        print(image_id)
        found_images = [image for image in images_data['images'] if image['id'] == image_id]

        if found_images:
            print("Images found:")
            for image in found_images:
                file_name = image['file_name']

        image_to_check = DETR+"/train/" +file_name

        image_to_check = cv2.imread(image_to_check)

        if positive:
            selected_row = train_df[train_df['slice'] == str(file_name)][['x_min','y_min','x_max','y_max']].iloc[0]

            xmin = selected_row['x_min']
            xmax = selected_row['x_max']
            ymin = selected_row['y_min']
            ymax = selected_row['y_max']

            cv2.rectangle(image_to_check, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            sv.show_frame_in_notebook(image_to_check, (4, 4))

        else:
            sv.show_frame_in_notebook(image_to_check, (4, 4))

In [ ]:
categories = TRAIN_DATASET.coco.cats
id2label = {k: v['name'] for k,v in categories.items()}
print(id2label)

{1: 'lung_cancer', 2: 'lymphoma', 3: 'melanoma'}


In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    # DETR authors employ various image sizes during training, making it not possible
    # to directly batch together images. Hence they pad the images to the biggest
    # resolution in a given batch, and create a corresponding binary pixel_mask
    # which indicates which pixels are real/which are padding
    pixel_values = [item[0] for item in batch]
    encoding = image_processor.pad(pixel_values, return_tensors="pt")
    labels = [item[1] for item in batch]
    return {
        'pixel_values': encoding['pixel_values'],
        'pixel_mask': encoding['pixel_mask'],
        'labels': labels
    }

BATCH_SIZE = 8
pin_memory = True
num_workers = 12

TRAIN_DATALOADER = DataLoader(dataset=TRAIN_DATASET, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True, pin_memory=pin_memory, num_workers = num_workers)
VAL_DATALOADER = DataLoader(dataset=VAL_DATASET, collate_fn=collate_fn, batch_size=BATCH_SIZE,pin_memory=pin_memory,num_workers = num_workers)
TEST_DATALOADER = DataLoader(dataset=TEST_DATASET, collate_fn=collate_fn, batch_size=BATCH_SIZE, pin_memory=pin_memory, num_workers = num_workers)

## Train model with PyTorch Lightning


In [ ]:
from datetime import date
import torch
import wandb
from pytorch_lightning.loggers import WandbLogger
import pytorch_lightning as pl

In [ ]:
CHECKPOINT

'microsoft/conditional-detr-resnet-50'

In [ ]:
from transformers import ConditionalDetrForObjectDetection
import torch

class Detr(pl.LightningModule):

    def __init__(self, lr, lr_backbone, weight_decay):
        super().__init__()
        self.model = ConditionalDetrForObjectDetection.from_pretrained(
            pretrained_model_name_or_path=CHECKPOINT,
            num_labels=len(id2label) + 1, ## WE need to add an additional class as no object class as per DETR official.the model will throw an error if it's not provided
            ignore_mismatched_sizes=True
        )

        self.lr = lr
        self.lr_backbone = lr_backbone
        self.weight_decay = weight_decay

    def forward(self, pixel_values, pixel_mask):
        return self.model(pixel_values=pixel_values, pixel_mask=pixel_mask)

    def common_step(self, batch, batch_idx):
        pixel_values = batch["pixel_values"]
        pixel_mask = batch["pixel_mask"]
        labels = [{k: v.to(self.device) for k, v in t.items()} for t in batch["labels"]]

        outputs = self.model(pixel_values=pixel_values, pixel_mask=pixel_mask, labels=labels)

        loss = outputs.loss
        loss_dict = outputs.loss_dict

        return loss, loss_dict

    def training_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)
        # logs metrics for each training_step, and the average across the epoch
        self.log("training_loss", loss)
        for k,v in loss_dict.items():
            self.log("train_" + k, v.item())

        return loss

    def validation_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)
        self.log("validation/loss", loss)
        for k, v in loss_dict.items():
            self.log("validation_" + k, v.item())

        return loss

    def configure_optimizers(self):

        # References
        # - https://github.com/facebookresearch/detr/blob/3af9fa878e73b6894ce3596450a8d9b89d918ca9/main.py#L22-L23
        # - https://github.com/facebookresearch/detr/blob/3af9fa878e73b6894ce3596450a8d9b89d918ca9/main.py#L131-L139
        param_dicts = [
            {
                "params": [p for n, p in self.named_parameters() if "backbone" not in n and p.requires_grad]},
            {
                "params": [p for n, p in self.named_parameters() if "backbone" in n and p.requires_grad],
                "lr": self.lr_backbone,
            },
        ]
        return torch.optim.AdamW(param_dicts, lr=self.lr, weight_decay=self.weight_decay)

    def train_dataloader(self):
        return TRAIN_DATALOADER

    def val_dataloader(self):
        return VAL_DATALOADER

%time

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [ ]:
model = Detr(lr=1e-4, lr_backbone=1e-5, weight_decay=1e-4)

batch = next(iter(TRAIN_DATALOADER))

outputs = model(pixel_values=batch['pixel_values'], pixel_mask=batch['pixel_mask'])

%time

Some weights of ConditionalDetrForObjectDetection were not initialized from the model checkpoint at microsoft/conditional-detr-resnet-50 and are newly initialized because the shapes did not match:
- class_labels_classifier.weight: found shape torch.Size([91, 256]) in the checkpoint and torch.Size([4, 256]) in the model instantiated
- class_labels_classifier.bias: found shape torch.Size([91]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs


In [ ]:
# wandb.finish()
# os.environ["WANDB_NOTEBOOK_NAME"] = 'conditional-detr-negative-CT'
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
!nvidia-smi

Wed Nov 29 05:07:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA L4           On   | 00000000:00:03.0 Off |                    0 |
| N/A   68C    P0    32W /  72W |    872MiB / 23034MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Model run for CT imags with negative data set included

In [ ]:
from pytorch_lightning import Trainer

print(" total number of classes ", outputs.logits.shape[2])

print("Annotations File Used ", ANNOTATION_FILE_NAME)

assert torch.cuda.is_available() == True

wandb_logger = WandbLogger(log_model="all", project="FINAL-DETR-STACKED")

today = str(date.today())

# settings
MAX_EPOCHS =  20

model_name = f'{today}-conditional-detr-checkpoint-epochs-{MAX_EPOCHS}-stacked-all.pkl'

MODEL_PATH = os.path.join(CHECKPOINT_PATH,model_name)

print(" MODEL RUN DATE : >>>>>>>>>>>> ", str(today))

print(" MODEL MODEL_PATH : >>>>>>>>>>>> ", str(MODEL_PATH))


# pytorch_lightning >= 2.0.0
trainer = Trainer(logger=wandb_logger,
                  devices=1,
                  accelerator="gpu",
                  max_epochs=MAX_EPOCHS,
                  gradient_clip_val=0.1,
                  accumulate_grad_batches=8,
                  log_every_n_steps=10)

trainer.fit(model)

model.model.save_pretrained(MODEL_PATH)

wandb.finish()

%time

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                              | Params
------------------------------------------------------------
0 | model | ConditionalDetrForObjectDetection | 43.4 M
------------------------------------------------------------
43.2 M    Trainable params
222 K     Non-trainable params
43.4 M    Total params
173.584   Total estimated model params size (MB)


 total number of classes  4
Annotations File Used  _annotations_stacked_all.coco.json
 MODEL RUN DATE : >>>>>>>>>>>>  2023-11-29
 MODEL MODEL_PATH : >>>>>>>>>>>>  /mnt/disks/storage/vk2501/checkpoints/2023-11-29-conditional-detr-checkpoint-epochs-20-stacked-all.pkl


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Validation: |          | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Validation: |          | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

Detr(
  (model): ConditionalDetrForObjectDetection(
    (model): ConditionalDetrModel(
      (backbone): ConditionalDetrConvModel(
        (conv_encoder): ConditionalDetrConvEncoder(
          (model): FeatureListNet(
            (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            (bn1): ConditionalDetrFrozenBatchNorm2d()
            (act1): ReLU(inplace=True)
            (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
            (layer1): Sequential(
              (0): Bottleneck(
                (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (bn1): ConditionalDetrFrozenBatchNorm2d()
                (act1): ReLU(inplace=True)
                (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn2): ConditionalDetrFrozenBatchNorm2d()
                (drop_block): Identity()
                (act2): ReLU(inplace=T

## Evaluation on test dataset

In [ ]:
model.to(DEVICE)

Detr(
  (model): ConditionalDetrForObjectDetection(
    (model): ConditionalDetrModel(
      (backbone): ConditionalDetrConvModel(
        (conv_encoder): ConditionalDetrConvEncoder(
          (model): FeatureListNet(
            (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            (bn1): ConditionalDetrFrozenBatchNorm2d()
            (act1): ReLU(inplace=True)
            (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
            (layer1): Sequential(
              (0): Bottleneck(
                (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (bn1): ConditionalDetrFrozenBatchNorm2d()
                (act1): ReLU(inplace=True)
                (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn2): ConditionalDetrFrozenBatchNorm2d()
                (drop_block): Identity()
                (act2): ReLU(inplace=T

In [ ]:
model.model.save_pretrained("model_stacked.pkl")

In [ ]:
def convert_to_xywh(boxes):
    xmin, ymin, xmax, ymax = boxes.unbind(1)
    return torch.stack((xmin, ymin, xmax - xmin, ymax - ymin), dim=1)

def prepare_for_coco_detection(predictions):
    coco_results = []
    for original_id, prediction in predictions.items():
        if len(prediction) == 0:
            continue

        boxes = prediction["boxes"]
        boxes = convert_to_xywh(boxes).tolist()
        scores = prediction["scores"].tolist()
        labels = prediction["labels"].tolist()

        coco_results.extend(
            [
                {
                    "image_id": original_id,
                    "category_id": labels[k],
                    "bbox": box,
                    "score": scores[k],
                }
                for k, box in enumerate(boxes)
            ]
        )
    return coco_results

In [ ]:
from coco_eval import CocoEvaluator
from tqdm.notebook import tqdm

import numpy as np

model.eval()

TEST_THRESHOLD = 0.5

evaluator = CocoEvaluator(coco_gt=TEST_DATASET.coco, iou_types=["bbox"])

print("Running evaluation...")
empty_data = []
for idx, batch in enumerate(tqdm(TEST_DATALOADER)):
    pixel_values = batch["pixel_values"].to(DEVICE)
    pixel_mask = batch["pixel_mask"].to(DEVICE)
    labels = [{k: v.to(DEVICE) for k, v in t.items()} for t in batch["labels"]]

    with torch.no_grad():
      outputs = model(pixel_values=pixel_values, pixel_mask=pixel_mask)

    orig_target_sizes = torch.stack([target["orig_size"] for target in labels], dim=0)
    results = image_processor.post_process_object_detection(outputs, target_sizes=orig_target_sizes, threshold = TEST_THRESHOLD)

    predictions = {target['image_id'].item(): output for target, output in zip(labels, results)}

    predictions = prepare_for_coco_detection(predictions)
    if len(predictions)>0:

        evaluator.update(predictions)
    else:
        empty_data.append(idx)
        print(idx)
evaluator.synchronize_between_processes()
evaluator.accumulate()
evaluator.summarize()

Running evaluation...


  0%|          | 0/495 [00:00<?, ?it/s]

275
332
370
371
372
373
374
375
376
377
378
381
382
383
384
385
386
387
388
390
391
392
393
395
396
398
399
400
401
402
403
404
405
406
407
409
410
411
412
414
415
416
417
418
419
420
421
422
423
424
425
426
429
430
431
432
433
434
436
437
438
440
442
443
446
447
448
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
490
491
492
494
Accumulating evaluation results...
DONE (t=0.15s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.708
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.307
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.277
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.491
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.641
 Average Rec

In [ ]:
len(empty_data)

81

## Inference on test dataset

Let's visualize the predictions of DETR on the first image of the validation set.

In [ ]:
image_path

'/mnt/disks/storage/vk2501/data/PETCT_stacked_DETR/test/PETCT_0cda25453b_PETCTstacked_axial_353.jpg'

In [2]:
import random
import cv2
import numpy as np

TEST_THRESHOLD= 0.5

model.eval()

for i in range(0,5):
                # utils
            categories = TEST_DATASET.coco.cats
            id2label = {k: v['name'] for k,v in categories.items()}
            box_annotator = sv.BoxAnnotator()

            # select random image
            image_ids = TEST_DATASET.coco.getImgIds()
            image_id = random.choice(image_ids)
            print('Image #{}'.format(image_id))

            # load image and annotatons
            image = TEST_DATASET.coco.loadImgs(image_id)[0]
            annotations = TEST_DATASET.coco.imgToAnns[image_id]
            image_path = os.path.join(TEST_DATASET.root, image['file_name'])
            image = cv2.imread(image_path)

            # annotate
            detections = sv.Detections.from_coco_annotations(coco_annotation=annotations)
            labels = [f"{id2label[class_id]}" for _, _, class_id, _ in detections]
            frame = box_annotator.annotate(scene=image.copy(), detections=detections, labels=labels)

            print('ground truth')
            %matplotlib inline
            sv.show_frame_in_notebook(frame, (4, 4))

            # inference
            with torch.no_grad():

                # load image and predict
                inputs = image_processor(images=image, return_tensors='pt').to(DEVICE)
                outputs = model(**inputs)

                # post-process
                target_sizes = torch.tensor([image.shape[:2]]).to(DEVICE)
                results = image_processor.post_process_object_detection(
                    outputs=outputs,
                    threshold=TEST_THRESHOLD,
                    target_sizes=target_sizes
                )[0]

            # annotate
            detections = sv.Detections.from_transformers(transformers_results=results)
            labels = [f"{id2label[class_id]} {confidence:.2f}" for _, confidence, class_id, _ in detections]
            frame = box_annotator.annotate(scene=image.copy(), detections=detections, labels=labels)

            print('detections')
            %matplotlib inline
            sv.show_frame_in_notebook(frame, (4, 4))

#### 10 epochs RUN

In [ ]:
!nvidia-smi

Tue Nov  7 22:05:10 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      On  | 00000000:00:03.0 Off |                    0 |
| N/A   52C    P0              29W /  72W |   2434MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Save and load model

In [ ]:
MODEL_PATH = 'model_stacked.pkl'

### 0.5 threshold

In [ ]:

reloaded_model = ConditionalDetrForObjectDetection.from_pretrained(MODEL_PATH)
reloaded_model.to(DEVICE)

ConditionalDetrForObjectDetection(
  (model): ConditionalDetrModel(
    (backbone): ConditionalDetrConvModel(
      (conv_encoder): ConditionalDetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): ConditionalDetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): ConditionalDetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): ConditionalDetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (

In [ ]:
from coco_eval import CocoEvaluator
from tqdm.notebook import tqdm

print("STACKED RELOADED MODEL")
import numpy as np

reloaded_model.eval()

evaluator = CocoEvaluator(coco_gt=TEST_DATASET.coco, iou_types=["bbox"])

not_found_preds = []
print("Running evaluation...")

for idx, batch in enumerate(tqdm(TEST_DATALOADER)):
    pixel_values = batch["pixel_values"].to(DEVICE)
    pixel_mask = batch["pixel_mask"].to(DEVICE)
    labels = [{k: v.to(DEVICE) for k, v in t.items()} for t in batch["labels"]]

    with torch.no_grad():
      outputs = reloaded_model(pixel_values=pixel_values, pixel_mask=pixel_mask)

    orig_target_sizes = torch.stack([target["orig_size"] for target in labels], dim=0)
    results = image_processor.post_process_object_detection(outputs, target_sizes=orig_target_sizes)

    predictions = {target['image_id'].item(): output for target, output in zip(labels, results)}
    predictions = prepare_for_coco_detection(predictions)
    if len(predictions) > 0 :
        evaluator.update(predictions)
    else:
            not_found_preds.append(idx)
            print(idx)

evaluator.synchronize_between_processes()
evaluator.accumulate()
evaluator.summarize()

STACKED RELOADED MODEL
Running evaluation...


  0%|          | 0/495 [00:00<?, ?it/s]

275
332
370
371
372
373
374
375
376
377
378
381
382
383
384
385
386
387
388
390
391
392
393
395
396
398
399
400
401
402
403
404
405
406
407
409
410
411
412
414
415
416
417
418
419
420
421
422
423
424
425
426
429
430
431
432
433
434
436
437
438
440
442
443
446
447
448
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
490
491
492
494
Accumulating evaluation results...
DONE (t=0.15s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.708
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.307
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.277
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.491
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.641
 Average Rec

In [ ]:
print("Predictions not found ")
len(not_found_preds)

Predictions not found 


110

### 0.2 Threshold

In [ ]:
MODEL_PATH = 'model_stacked.pkl'
reloaded_model = ConditionalDetrForObjectDetection.from_pretrained(MODEL_PATH)
reloaded_model.to(DEVICE)

ConditionalDetrForObjectDetection(
  (model): ConditionalDetrModel(
    (backbone): ConditionalDetrConvModel(
      (conv_encoder): ConditionalDetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): ConditionalDetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): ConditionalDetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): ConditionalDetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (

In [ ]:
from coco_eval import CocoEvaluator
from tqdm.notebook import tqdm


import numpy as np

THRESHOLD_SCORE = 0.2
evaluator = CocoEvaluator(coco_gt=TEST_DATASET.coco, iou_types=["bbox"])

print(f"STACKED DETR ALL THRESHOLD {THRESHOLD_SCORE}")

print("Running evaluation...")
not_found = []

for idx, batch in enumerate(tqdm(TEST_DATALOADER)):
    pixel_values = batch["pixel_values"].to(DEVICE)
    pixel_mask = batch["pixel_mask"].to(DEVICE)
    labels = [{k: v.to(DEVICE) for k, v in t.items()} for t in batch["labels"]]

    with torch.no_grad():
      outputs = reloaded_model(pixel_values=pixel_values, pixel_mask=pixel_mask)

    orig_target_sizes = torch.stack([target["orig_size"] for target in labels], dim=0)
    results = image_processor.post_process_object_detection(outputs,threshold=THRESHOLD_SCORE, target_sizes=orig_target_sizes, )

    predictions = {target['image_id'].item(): output for target, output in zip(labels, results)}
    predictions = prepare_for_coco_detection(predictions)

    if len(predictions) > 0 :
            evaluator.update(predictions)
    else:
        print(idx)
        not_found.append(idx)

evaluator.synchronize_between_processes()
evaluator.accumulate()
evaluator.summarize()

STACKED DETR ALL THRESHOLD 0.2
Running evaluation...


  0%|          | 0/495 [00:00<?, ?it/s]

374
376
381
384
393
398
404
433
434
436
438
472
485
491
Accumulating evaluation results...
DONE (t=0.27s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.285
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.598
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.234
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.212
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.424
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.611
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.385
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.406
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.406
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.333
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.